<a href="https://colab.research.google.com/github/gp-rooney/GPR-Website/blob/main/Copy_of_Correlations_Beta_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pandas numpy statsmodels yfinance tqdm lxml


In [ ]:
import pandas as pd, numpy as np, statsmodels.api as sm
import yfinance as yf
from tqdm import tqdm
from google.colab import files
import requests

In [ ]:
# ==========
# 1) Tell the script where your file is
# ==========
import os

# Option 1: If you UPLOADED the CSV via the Colab file upload button:
#   from google.colab import files
#   uploaded = files.upload()  # choose your CSV
#   TICKER_PATH = list(uploaded.keys())[0]

# Option 2: If the file is already in your Colab working dir (e.g., after uploading):
#   Replace with your actual filename (case-sensitive)
TICKER_PATH = "S&P 500 Tickers - Sheet1.csv"  # change if your file has a different name

# Option 3: If the file lives in Google Drive:
#   from google.colab import drive
#   drive.mount('/content/drive')
#   TICKER_PATH = "/content/drive/MyDrive/path/to/your/file.csv"

# Print current working directory and list files to help verify file path
print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir())

assert os.path.exists(TICKER_PATH), f"File not found: {TICKER_PATH}"

# Read CSV or Excel
if TICKER_PATH.lower().endswith(".csv"):
    df_ticks = pd.read_csv(TICKER_PATH)
else:
    # pip install openpyxl if using .xlsx
    # !pip -q install openpyxl
    df_ticks = pd.read_excel(TICKER_PATH)

# Find the ticker column (accept common names)
ticker_col = None
for c in df_ticks.columns:
    if c.strip().lower() in ("symbol", "ticker", "tickers"):
        ticker_col = c
        break
if ticker_col is None:
    raise ValueError("Could not find a 'Symbol' or 'Ticker' column in your file.")

# Clean tickers, handle BRK.B -> BRK-B, etc.
tickers = (
    df_ticks[ticker_col]
    .astype(str)
    .str.strip()
    .str.replace(".", "-", regex=False)
    .tolist()
)



Current working directory: /content
Files in current directory: ['.config', 'sp500_weekly_r2.csv', 'S&P 500 Tickers - Sheet1.csv', 'sample_data']


In [ ]:
# Define the start and end dates for the data download
START = "2022-01-01"  # Example start date
END = "2025-11-07"  # Example end date - you can adjust this

In [ ]:
print(f"Data will be downloaded from {START} to {END}.")

Data will be downloaded from 2022-01-01 to 2025-11-07.


In [ ]:
# 2) Download adjusted close prices
# Define bench_symbol before it is used
bench_symbol = "^SP500TR" # You can change this to another benchmark if needed

data = yf.download(tickers, start=START, end=END, auto_adjust=True, progress=False)

# Check the structure of the downloaded data
print(data.head())

# Assuming the data is a DataFrame with a single level column index after auto_adjust=True
# If the data has multiple levels, you might need to adjust how you access "Adj Close"
adj = data["Close"]


# Fallback if ^SP500TR is unavailable in your region
if bench_symbol not in adj.columns or adj[bench_symbol].dropna().empty:
    bench_symbol = "SPY"
    if bench_symbol not in adj.columns:
        adj2 = yf.download([bench_symbol], start=START, end=END, auto_adjust=True, progress=False)["Close"]
        adj = adj.join(adj2, how="outer")

# 3) Weekly series
def weekly(series):
    return series.resample("W-FRI").mean() if USE_WEEKLY_AVG_PRICE else series.resample("W-FRI").last()

bench_w = weekly(adj[bench_symbol]).dropna()
bench_r = np.log(bench_w).diff().dropna()

rows = []
for t in tqdm(tickers):
    if t not in adj.columns:
        continue
    s = weekly(adj[t]).dropna()
    r = np.log(s).diff()
    df = pd.concat([r.rename("stock"), bench_r.rename("mkt")], axis=1, join="inner").dropna()
    if len(df) < MIN_WEEKS:
        continue
    X = sm.add_constant(df["mkt"])
    y = df["stock"]
    model = sm.OLS(y, X).fit()
    beta = model.params["mkt"]
    r2 = model.rsquared
    corr = np.sign(beta) * np.sqrt(r2)  # signed stock–market correlation implied by the one-factor fit
    n = int(model.nobs)
    rows.append({"ticker": t, "beta": beta, "r2": r2, "corr": corr, "n_obs": n})

out = pd.DataFrame(rows).sort_values("r2", ascending=False)
out.to_csv("sp500_weekly_r2.csv", index=False)

# === Averages you asked for ===
avg_corr = out["corr"].mean()
avg_abs_corr = out["corr"].abs().mean()
w_avg_corr = np.average(out["corr"], weights=out["n_obs"])
w_avg_abs_corr = np.average(out["corr"].abs(), weights=out["n_obs"])

print(f"Wrote sp500_weekly_r2.csv with {len(out)} tickers.")
print(f"Average signed correlation: {avg_corr:.3f}")
print(f"Average absolute correlation: {avg_abs_corr:.3f}")
print(f"Weighted (by weeks) signed correlation: {w_avg_corr:.3f}")
print(f"Weighted (by weeks) absolute correlation: {w_avg_abs_corr:.3f}")

files.download("sp500_weekly_r2.csv")
display(out)

Price            Close                                                  \
Ticker               A        AAPL        ABBV        ABNB         ABT   
Date                                                                     
2022-01-03  152.320068  178.443130  116.779289  172.679993  128.996140   
2022-01-04  147.170685  176.178406  116.555099  170.800003  125.962357   
2022-01-05  144.649506  171.492096  117.167351  162.250000  125.396385   
2022-01-06  145.155701  168.629303  116.615440  159.750000  125.377823   
2022-01-07  141.291260  168.795929  116.313644  166.050003  125.767540   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2022-01-03  42.362530  383.711914  564.369995  165.624542  60.999535  ...   
2022-01-04  42.914051  380.969849  554.000000  164.128647  62.131653  ...   
2022-01-05  42.410072 

100%|██████████| 503/503 [00:04<00:00, 114.74it/s]

Wrote sp500_weekly_r2.csv with 501 tickers.
Average signed correlation: 0.508
Average absolute correlation: 0.508
Weighted (by weeks) signed correlation: 0.509
Weighted (by weeks) absolute correlation: 0.509


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,ticker,beta,r2,corr,n_obs
64,BLK,1.373455,0.650920,0.806796,200
323,MCO,1.267042,0.620470,0.787699,200
33,APH,1.258760,0.610575,0.781393,200
274,KKR,1.764937,0.605223,0.777961,200
430,TROW,1.457080,0.604185,0.777293,200
...,...,...,...,...,...
82,CPB,0.161498,0.016383,0.127997,200
462,UNH,0.254589,0.015327,0.123800,200
147,DG,0.269168,0.013794,0.117447,200
318,MOH,0.231275,0.013001,0.114024,200
